# PASSIVE-FRAUD-MESSAGE-DETECTION-NEW-STRATEGIES-COMBINING-DDPG-AND-RAG-TECHNOLOGIESA

### 本代碼不提供api key，請自行填入google api key以及openai_api_key

In [1]:
!pip install kaggle gradio tensorflow keras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.5 MB/s eta 

In [2]:
from google.colab import files
files.upload()  # 上傳kaggle.json


KeyboardInterrupt: 

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

!kaggle datasets download -d shalini2810/input-file
!unzip input-file.zip


Dataset URL: https://www.kaggle.com/datasets/shalini2810/input-file
License(s): other
100% 211k/211k [00:00<00:00, 494kB/s]
100% 211k/211k [00:00<00:00, 494kB/s]
Archive:  input-file.zip
  inflating: spam.csv                


In [3]:
!pip install langchain openai faiss-gpu tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
w

In [5]:
!pip install kaggle gradio tensorflow keras
!pip install requests langchain faiss-cpu langchain-community

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
import gradio as gr
import requests
import json
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chains import LLMChain
# 讀取資料集
df = pd.read_csv('spam.csv', encoding='latin-1')

# 刪除多餘的欄位
df = df[['class', 'message']]
df.columns = ['label', 'text']
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# 分割訓練集和測試集
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=3000)
X_train = vectorizer.fit_transform(train_df['text']).toarray()
y_train = train_df['label'].values
X_test = vectorizer.transform(test_df['text']).toarray()
y_test = test_df['label'].values

# 建立DDPG模型
def build_actor(state_dim):
    state_input = Input(shape=(state_dim,))
    x = Dense(128, activation='relu')(state_input)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=state_input, outputs=output)
    return model

def build_critic(state_dim):
    state_input = Input(shape=(state_dim,))
    action_input = Input(shape=(1,))
    concat = Concatenate()([state_input, action_input])
    x = Dense(128, activation='relu')(concat)
    x = Dense(64, activation='relu')(x)
    output = Dense(1)(x)
    model = Model(inputs=[state_input, action_input], outputs=output)
    return model

state_dim = X_train.shape[1]
actor_model = build_actor(state_dim)
critic_model = build_critic(state_dim)

actor_optimizer = Adam(learning_rate=0.001)
critic_optimizer = Adam(learning_rate=0.002)

actor_model.compile(optimizer=actor_optimizer, loss='binary_crossentropy')
critic_model.compile(optimizer=critic_optimizer, loss='mean_squared_error')

# 訓練DDPG模型
actor_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# 構建RAG系統
def fact_check_all_english(query, api_key):
    url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"
    params = {
        'query': query,
        'key': api_key,
        'pageSize': 50,  # 每页返回较多结果以增加找到英文结果的机会
    }
    results = []
    while True:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            claims = data.get('claims', [])
            english_claims = [claim for claim in claims if any(review.get('languageCode') == 'en' for review in claim.get('claimReview', []))]
            results.extend(english_claims)
            next_page_token = data.get('nextPageToken')
            if next_page_token:
                params['pageToken'] = next_page_token
            else:
                break
        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    filtered_results = []
    for claim in results:
        for review in claim.get('claimReview', []):
            if review.get('languageCode') == 'en':
                filtered_results.append({
                    'text': claim.get('text'),
                    'url': review.get('url'),
                    'title': review.get('title'),
                    'textualRating': review.get('textualRating')
                })

    return filtered_results

def save_results_to_json(results, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(results, file, ensure_ascii=False, indent=4)

# 使用您的API凭证和查询关键词
api_key = "your_api_key"
query = "scam"  # 这里可以设置为您想查询的任何关键词

all_english_results = fact_check_all_english(query, api_key)
save_results_to_json(all_english_results, 'fact_check_results.json')

def load_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return json.load(file)

data = load_json('fact_check_results.json')

documents = [
    Document(
        page_content=item['text'],
        metadata={
            'url': item['url'],
            'title': item['title'],
            'textualRating': item['textualRating']
        }
    )
    for item in data
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key="your_api_key")
vector_store = FAISS.from_documents(split_documents, embeddings)

api_key = "your_api_key"
openai_api = OpenAI(api_key=api_key)

prompt_template = """
You are a fact-checking assistant. You are given a user query and you need to provide accurate information based on the available documents.

User Query: {query}

Relevant Information:
{documents}

For the most relevant information, provide the title of the document, the URL, and an explanation the most relevant document.

Please provide a detailed response based on the relevant information.
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["query", "documents"])

llm_chain = LLMChain(llm=openai_api, prompt=prompt)

class RetrievalAugmentedGenerator:
    def __init__(self, vector_store, generator, num_retrieved_documents=3):
        self.vector_store = vector_store
        self.generator = generator
        self.num_retrieved_documents = num_retrieved_documents

    def __call__(self, query):
        retrieved_docs = self.vector_store.similarity_search(query, k=self.num_retrieved_documents)
        documents = "\n\n".join([
            f"Title: {doc.metadata['title']}\nURL: {doc.metadata['url']}\nText: {doc.page_content}"
            for doc in retrieved_docs
        ])
        response = self.generator.run(query=query, documents=documents)
        return response

rag_system = RetrievalAugmentedGenerator(
    vector_store=vector_store,
    generator=llm_chain,
    num_retrieved_documents=3
)

def query_rag_system(query):
    result = rag_system(query)
    return result


Epoch 1/10
140/140 [==============================] - 4s 16ms/step - loss: 0.2493 - val_loss: 0.0905
Epoch 2/10
140/140 [==============================] - 1s 10ms/step - loss: 0.0340 - val_loss: 0.0741
Epoch 3/10
140/140 [==============================] - 2s 17ms/step - loss: 0.0082 - val_loss: 0.0800
Epoch 4/10
140/140 [==============================] - 2s 18ms/step - loss: 0.0032 - val_loss: 0.0983
Epoch 5/10
140/140 [==============================] - 2s 17ms/step - loss: 0.0012 - val_loss: 0.0994
Epoch 6/10
140/140 [==============================] - 3s 20ms/step - loss: 6.1853e-04 - val_loss: 0.1074
Epoch 7/10
140/140 [==============================] - 3s 21ms/step - loss: 3.5921e-04 - val_loss: 0.1130
Epoch 8/10
140/140 [==============================] - 2s 17ms/step - loss: 2.5179e-04 - val_loss: 0.1133
Epoch 9/10
140/140 [==============================] - 2s 14ms/step - loss: 1.7382e-04 - val_loss: 0.1170
Epoch 10/10
140/140 [==============================] - 3s 20ms/step - loss:

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain

In [52]:
feedback_data = {'states': [], 'actions': [], 'rewards': []}

def clear_input():
    return "", "", "", ""

def classify_message(message):
    state = vectorizer.transform([message]).toarray()
    action = actor_model.predict(state)[0][0]
    result = 'Spam' if action > 0.5 else 'Ham'
    if result == 'Spam':
        fact_check_result = query_rag_system(message)
        return result, fact_check_result, ""
    else:
        return result, "No fact check needed for ham messages.", ""

def feedback(result, message, like):
    state = vectorizer.transform([message]).toarray()
    reward = 1 if like == 'Like' else -1

    feedback_data['states'].append(state)
    feedback_data['actions'].append([1 if result == 'Spam' else 0])
    feedback_data['rewards'].append(reward)

    log = "Feedback received:\n"
    log += f"States: {feedback_data['states']}\n"
    log += f"Actions: {feedback_data['actions']}\n"
    log += f"Rewards: {feedback_data['rewards']}\n"

    if len(feedback_data['states']) >= 10:
        states = np.vstack(feedback_data['states'])
        actions = np.array(feedback_data['actions'])
        rewards = np.array(feedback_data['rewards'])

        log += "Retraining model with feedback data...\n"
        target_q_values = rewards
        history = critic_model.fit([states, actions], target_q_values, epochs=1, verbose=1)
        feedback_data['states'], feedback_data['actions'], feedback_data['rewards'] = [], [], []

        log += f"Retraining loss: {history.history['loss']}\n"

    return "Feedback received. Thank you!", log

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=8):
          message_input = gr.Textbox(lines=2, placeholder="Enter your message here...", label="Message")
        with gr.Column(scale=1):
          classify_button = gr.Button("Classify")
          clear_button = gr.Button("Clear")
    with gr.Row():
      with gr.Column(scale=1):
        output = gr.Textbox(label="Result")
        feedback_log = gr.Textbox(label="Feedback Log")
      with gr.Column(scale=5):
        fact_check_output = gr.Textbox(label="Fact Check Result", lines=5)
    with gr.Row():
        like_button = gr.Button("Like")
        dislike_button = gr.Button("Dislike")

    classify_button.click(classify_message, inputs=message_input, outputs=[output, fact_check_output, feedback_log])
    like_button.click(feedback, inputs=[output, message_input, gr.State('Like')], outputs=feedback_log)
    dislike_button.click(feedback, inputs=[output, message_input, gr.State('Dislike')], outputs=feedback_log)
    clear_button.click(clear_input, outputs=[message_input, output, fact_check_output, feedback_log])

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://feb6a68007fc3aefc3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
